# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [3]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [4]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [5]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [6]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
display(prod_cust_pivot.head(3))
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
display(cust_prod_pivot.head(3))

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000521,0.0,0.0,...,0.000000,0.013034,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.011221,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000532,0.0,0.0,...,0.013298,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.013298,0.0


ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.015873,0.000000,0.000000,0.0,0.0,0.015873,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.015873,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.012987,0.0,0.0,0.000000,0.0,0.0,0.0,0.012987,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.015385,0.015385,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.015385,0.0,0.0,0.0,0.000000,0.000000,0.0


## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [7]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [8]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [9]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [10]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [11]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [12]:
# En este caso no hago la traspuesta de prod_cust_pivot en la funcion pdist, porque esa pivot table ya tiene los productos por filas
product_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)

product_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Appetizer - Sausage Rolls,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Apricots - Dried,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477


### Step 2: Get the products purchased for a specific customer of your choice.

In [13]:
prod_33 = pd.DataFrame(prod_cust_pivot[33])
prod_33 = prod_33[prod_33[33]!=0]
prod_33.head()

,33
ProductName,
Apricots - Dried,0.000491
Assorted Desserts,0.000475
Bandage - Flexible Neon,0.000426
"Bar Mix - Pina Colada, 355 Ml",0.000479
"Beans - Kidney, Canned",0.000496


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [14]:
products=[]
for e in prod_33.index:
    products.append(list(product_dist[e].sort_values(ascending=False)[1:6].index))
all_products = [e for lista in products for e in lista]
print(len(all_products))
for e in all_products:
    if e in prod_33.index:
        all_products.remove(e)
len(all_products)

295


260

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [15]:
times_product=[]
for e in all_products:
    times_product.append((e,all_products.count(e)))
max_times = sorted(set(times_product), key=lambda x: x[1],reverse=True)
max_times = list(max_times[i][0] for i in range(5))
max_times

['Sun - Dried Tomatoes',
 'Spinach - Baby',
 'Beef - Montreal Smoked Brisket',
 'Rosemary - Primerba, Paste',
 'Cookies - Assorted']

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [16]:
prod_recomm = {}
customers_ID = [e for e in prod_cust_pivot.columns]
for e in customers_ID:
    prod_ID = prod_cust_pivot[e][prod_cust_pivot[e]!=0]
    products=[]
    for product in prod_ID.index:
        products.append(list(product_dist[product].sort_values(ascending=False)[1:6].index))
    all_products = [prod for lista in products for prod in lista]
    for prod in all_products:
        if prod in prod_ID.index:
            all_products.remove(prod)
    times_product=[]
    for prod in all_products:
        times_product.append((prod,all_products.count(prod)))
    max_times = sorted(set(times_product), key=lambda x: x[1],reverse=True)
    max_times = list(max_times[i][0] for i in range(5))
    prod_recomm[e] = max_times

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [17]:
customer_recomm = pd.DataFrame(prod_recomm.values(),index=prod_recomm.keys())
customer_recomm

,0,1,2,3,4
33,Sun - Dried Tomatoes,Spinach - Baby,Beef - Montreal Smoked Brisket,"Rosemary - Primerba, Paste",Cookies - Assorted
200,"Rosemary - Primerba, Paste",Spinach - Baby,Oil - Shortening - All - Purpose,Rice - Jasmine Sented,Sun - Dried Tomatoes
264,"Rosemary - Primerba, Paste",Spinach - Baby,Sun - Dried Tomatoes,Oil - Shortening - All - Purpose,"Hickory Smoke, Liquid"
356,Spinach - Baby,Beef - Montreal Smoked Brisket,"Rosemary - Primerba, Paste",Veal - Osso Bucco,Sun - Dried Tomatoes
412,"Rosemary - Primerba, Paste",Sun - Dried Tomatoes,Oil - Shortening - All - Purpose,V8 - Berry Blend,"Yogurt - Blueberry, 175 Gr"
...,...,...,...,...,...
97928,V8 - Berry Blend,Sun - Dried Tomatoes,Spinach - Baby,"Rosemary - Primerba, Paste",Cod - Black Whole Fillet
98069,Spinach - Baby,"Hickory Smoke, Liquid",Beef - Montreal Smoked Brisket,"Rosemary - Primerba, Paste",V8 - Berry Blend
98159,V8 - Berry Blend,Spinach - Baby,Cookies - Assorted,Sun - Dried Tomatoes,"Chocolate - Semi Sweet, Calets"
98185,Oil - Shortening - All - Purpose,Spinach - Baby,Sun - Dried Tomatoes,Towels - Paper / Kraft,Cookies - Assorted


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [18]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [19]:
#Introduzco el nuevo cliente en la pivot table inicial
prod_cust_pivot["new_customer"] = pd.Series(new_customer)
prod_cust_pivot = prod_cust_pivot.fillna(0)
display(prod_cust_pivot.shape,prod_cust_pivot.head())

(452, 1001)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,new_customer
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,0.0,...,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0,0.0
Apricots - Dried,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,0.0,...,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [20]:
#Hago aquí la traspuesta porque en la función pdist, calcula las distancias por filas, así que necesitamos que los clientes
#están en las filas (se podría hacer en la propia función)
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.000491,0.000000,0.000000,0.0,0.0,0.000475,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000457,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000478,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000478,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.000472,0.000457,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000596,0.0,0.0,0.0,0.000000,0.000000,0.0
356,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000457,0.0
412,0.0,0.0,0.0,0.0,0.000491,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000434,0.000469,0.000596,0.0,0.0,0.0,0.000000,0.000000,0.0


In [21]:
#Normalizo los valores dentro de cada producto para que no sea dependiente de las cantidades totales compradas de cada uno
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)
display(cust_prod_pivot.head(3))

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,0.015191,0.000000,0.000000,0.0,0.0,0.014672,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.014135,0.0
200,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.012126,0.0,0.0,0.000000,0.0,0.0,0.0,0.012108,0.000000,0.0
264,0.0,0.0,0.0,0.0,0.000000,0.014075,0.013618,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.017776,0.0,0.0,0.0,0.000000,0.000000,0.0


In [22]:
#calculo la matriz de distancias
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,new_customer
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.853471,0.858742,0.853306,0.856845,0.851419,0.848198,0.847844,0.851494,0.847477,...,0.846215,0.849391,0.840335,0.851613,0.846271,0.851671,0.846575,0.855182,0.850362,0.750607
200,0.853471,1.000000,0.857755,0.856816,0.856528,0.852537,0.862793,0.850338,0.857746,0.856018,...,0.856246,0.862707,0.853879,0.856899,0.854896,0.869280,0.858026,0.861805,0.856897,0.749955
264,0.858742,0.857755,1.000000,0.852499,0.860711,0.850284,0.854831,0.846398,0.855225,0.847416,...,0.850261,0.853136,0.848543,0.858594,0.860244,0.859258,0.853900,0.865122,0.852414,0.750890
356,0.853306,0.856816,0.852499,1.000000,0.853916,0.850489,0.853008,0.851379,0.848804,0.845225,...,0.848272,0.852503,0.847467,0.851569,0.851808,0.850509,0.854184,0.860359,0.854677,0.745911
412,0.856845,0.856528,0.860711,0.853916,1.000000,0.854357,0.857896,0.857019,0.856104,0.854800,...,0.862552,0.855099,0.849638,0.857780,0.856193,0.854690,0.855880,0.865306,0.861828,0.759886


In [23]:
similar_cust = list(cust_dist["new_customer"].sort_values(ascending=False)[1:].head().index)
similar_cust

[64348, 39813, 36740, 412, 69397]

In [24]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
sim_cust_prod.head()

,CustomerID,ProductName,Quantity
254,412,Apricots - Dried,1
255,412,Banana - Leaves,1
256,412,Bananas,1
257,412,Beans - Wax,1
258,412,Beef - Rib Eye Aaa,1


In [25]:
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
grouped.head()

,Quantity
ProductName,
Anchovy Paste - 56 G Tube,17
"Appetizer - Mini Egg Roll, Shrimp",18
Appetizer - Mushroom Tart,17
Apricots - Dried,12
Apricots Fresh,11


In [26]:
ranked_products = grouped.sort_values('Quantity', ascending=False)
ranked_products.head()

,Quantity
ProductName,
Snapple Lemon Tea,153
Soup - Campbells Tomato Ravioli,63
Jagermeister,57
Bouq All Italian - Primerba,49
Spinach - Baby,47


In [27]:
prod_newCust = pd.DataFrame(prod_cust_pivot["new_customer"])
prod_newCust

,new_customer
ProductName,
Anchovy Paste - 56 G Tube,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0
Appetizer - Mushroom Tart,0.0
Appetizer - Sausage Rolls,0.0
Apricots - Dried,0.0
...,...
Yeast Dry - Fermipan,0.0
Yoghurt Tubes,0.0
"Yogurt - Blueberry, 175 Gr",0.0


In [28]:
merge_newCust = prod_newCust.merge(right=ranked_products, how='left', on="ProductName")
merge_newCust.head()

,new_customer,Quantity
ProductName,,
Anchovy Paste - 56 G Tube,0.0,17.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,18.0
Appetizer - Mushroom Tart,0.0,17.0
Appetizer - Sausage Rolls,0.0,NaN
Apricots - Dried,0.0,12.0


In [29]:
merge_newCust = merge_newCust[merge_newCust['new_customer']==0].sort_values("Quantity",ascending=False)
merge_newCust.head()

,new_customer,Quantity
ProductName,,
Soup - Campbells Tomato Ravioli,0.0,63.0
Jagermeister,0.0,57.0
Bouq All Italian - Primerba,0.0,49.0
Foam Cup 6 Oz,0.0,44.0
Milk - 2%,0.0,41.0


In [30]:
recomm_newCust = list(merge_newCust.index[:5])
recomm_newCust

['Soup - Campbells Tomato Ravioli',
 'Jagermeister',
 'Bouq All Italian - Primerba',
 'Foam Cup 6 Oz',
 'Milk - 2%']

NOTA: para recomendar por similitud con otros usuarios busco los 5 usuarios (o el número que quiera) más similares al cliente según la matriz de distancia de clientes. Extraigo todos los productos que han consumido esos 5 usuarios y hago una tabla con los productos y el total adquirido de esos productos por los 5 usuarios. Hago un merge de esa tabla con la tabla de cantidad consumida de los productos por el cliente que estamos anañizando. Me quedo solo con los productos que el cliente no ha consumido nunca y ordeno por cantidad consumida por los otros clientes. De ahí saco el top 5 (por ejemplo) de productos a recomendar.

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [31]:
#Uso la pivot table prod_cust_pivot que ya tiene los productos en las filas y así no tengo que trasponer más tarde en pdist
#Normalizo los valores dentro de cada cliente para que no sea dependiente de las cantidades compradas por cada cliente
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
display(prod_cust_pivot.shape,prod_cust_pivot.head())

(452, 1001)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,new_customer
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,0.0,...,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0,0.0
Apricots - Dried,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,0.0,...,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [32]:
product_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'euclidean'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)

product_dist.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.881127,0.882219,0.884604,0.890449,0.879859,0.885648,0.880852,0.886501,0.890573,...,0.884117,0.888124,0.886567,0.879578,0.875696,0.878427,0.886163,0.886564,0.887956,0.878223
"Appetizer - Mini Egg Roll, Shrimp",0.881127,1.000000,0.883877,0.880604,0.887930,0.881577,0.885241,0.890196,0.888646,0.883484,...,0.887569,0.887831,0.888033,0.881019,0.878720,0.871915,0.888702,0.890837,0.887661,0.882616
Appetizer - Mushroom Tart,0.882219,0.883877,1.000000,0.884708,0.883266,0.884124,0.886495,0.886072,0.887707,0.881019,...,0.886796,0.885968,0.885056,0.887445,0.882789,0.879286,0.890062,0.889960,0.889397,0.881077
Appetizer - Sausage Rolls,0.884604,0.880604,0.884708,1.000000,0.885025,0.885020,0.881992,0.883090,0.880745,0.881226,...,0.882793,0.886507,0.884984,0.886515,0.877195,0.868976,0.885470,0.884688,0.883661,0.877701
Apricots - Dried,0.890449,0.887930,0.883266,0.885025,1.000000,0.886792,0.887542,0.889058,0.886234,0.887162,...,0.886223,0.889001,0.890087,0.887478,0.878571,0.872542,0.892076,0.891209,0.889794,0.886477


In [33]:
#obtengo los productos comprados por el new_customer (variable ya creada en el ejercicio anterior)
prod_newCust
display(prod_newCust)
#Elimino los productos que el cliente no ha consumido nunca
prod_newCust = prod_newCust[prod_newCust["new_customer"]!=0]
prod_newCust.head()

,new_customer
ProductName,
Anchovy Paste - 56 G Tube,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0
Appetizer - Mushroom Tart,0.0
Appetizer - Sausage Rolls,0.0
Apricots - Dried,0.0
...,...
Yeast Dry - Fermipan,0.0
Yoghurt Tubes,0.0
"Yogurt - Blueberry, 175 Gr",0.0


,new_customer
ProductName,
Bandage - Flexible Neon,3.0
Beef - Montreal Smoked Brisket,4.0
Cookies - Assorted,3.0
Fenngreek Seed,1.0
Flavouring - Orange,3.0


In [35]:
# Busco los productos más similares a los consumidos por el new_customer (segun la matriz de similitud de productos)
products=[]
for e in prod_newCust.index:
    products.append(list(product_dist[e].sort_values(ascending=False)[1:6].index))
all_products = [e for lista in products for e in lista]
print(len(all_products))
#elimino aquellos que ya han sido consumidos por new_customer
for e in all_products:
    if e in prod_newCust.index:
        all_products.remove(e)
len(all_products)

60


30

In [39]:
#cuento las veces que se repiten los productos y ordeno
times_product=[]
for e in all_products:
    times_product.append((e,all_products.count(e)))
#ordeno los productos de más a menos veces repetidos (haciendo primero un set de la lista times_product)
max_times = sorted(set(times_product), key=lambda x: x[1],reverse=True)
#extraigo los 5 primeros en una lista
max_times = list(max_times[i][0] for i in range(5))
max_times

['French Pastry - Mini Chocolate',
 'Cookies - Assorted',
 'Bandage - Flexible Neon',
 'Spinach - Baby',
 'Beef - Montreal Smoked Brisket']

NOTA: para recomendar por producto busco los productos consumidos por el cliente (eliminando los que el cliente no ha consumido nunca). De cada producto consumido extraigo los 5 (el número que quiera) más próximos de la matriz de distancia de productos. Elimino de esa lista los que ya han sido consumidos por el cliente y cuento el número de veces que se están presentes los productos similares. Elijo el top 5 (por ejemplo) de los más repetidos de esa lista.